In [1]:
!pip install timm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
from tqdm.notebook import tqdm
import timm
import torch.optim.lr_scheduler as lr_scheduler
import math
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import pandas as pd
import torch
import numpy as np
# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.1 MB/s eta 0:00:00
cpu


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Pre-process the data

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [5]:
# Load all the data

import os
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

data_root = '/content/drive/MyDrive/mushroom_data_new'
batch_size = 32


# 1. Load all data using ImageFolder
full_dataset = ImageFolder(data_root, transform=transform)

# 2. Calculate lengths for train, validation, and test splits
total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size

# 3. Use random_split to split the datasets
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

def set_transform(dataset_subset, transform):
    dataset_subset.dataset.transform = transform
    return dataset_subset

val_dataset = set_transform(val_dataset, transform)
test_dataset = set_transform(test_dataset, transform)

# 4. Create DataLoaders for each set
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(len(trainloader))


279


In [6]:
batch_size = 32


# 1. Load all data using ImageFolder
full_dataset = ImageFolder(data_root, transform=transform)

# 2. Calculate lengths for train, validation, and test splits
total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size

# 3. Use random_split to split the datasets
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

def set_transform(dataset_subset, transform):
    dataset_subset.dataset.transform = transform
    return dataset_subset

train_dataset = set_transform(train_dataset, transform)
val_dataset = set_transform(val_dataset, transform)
test_dataset = set_transform(test_dataset, transform)

# 4. Create DataLoaders for each set
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(len(trainloader))

279


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Transformer-DR Encoder Block
class EncoderBlock(nn.Module):
    def __init__(self, d_model, nhead, feedforward_dim):
        super(EncoderBlock, self).__init__()
        self.multihead_attention = nn.MultiheadAttention(d_model, nhead)
        self.layer_norm1 = nn.LayerNorm(d_model)
        self.layer_norm2 = nn.LayerNorm(d_model)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, feedforward_dim),
            nn.GELU(),
            nn.Linear(feedforward_dim, d_model)
        )

    def forward(self, x):
        attn_out, _ = self.multihead_attention(x, x, x)
        x = self.layer_norm1(x + attn_out)
        ff_out = self.feed_forward(x)
        x = self.layer_norm2(x + ff_out)
        return x



# Hyperparameters
d_model = 512
nhead = 8
feedforward_dim = 2048
num_blocks = 6
input_dim = 784  # 28*28 = 784
learning_rate = 0.001

# Model
model = timm.create_model("vit_large_patch16_224", pretrained=True)

# Adjust the head of the model for mushroom classes
num_classes = len(full_dataset.classes)
model.head = torch.nn.Linear(in_features=model.head.in_features, out_features=num_classes)

# Training Loop
num_epochs = 2



for epoch in range(num_epochs):
    for batch in trainloader:
        inputs, label = batch
        outputs, _ = model(inputs)
        loss = criterion(outputs, inputs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

print("Finished Training!")


model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]